In [3]:
from datasets import load_dataset

Tinyds = load_dataset("roneneldan/TinyStories")

In [4]:
# Check the structure of the train and validation splits
print("Train split example:", Tinyds['train'][0])  # Print the first example of the train split
print("Validation split example:", Tinyds['validation'][0])  # Print the first example of the validation0split

# Check the length of the splits to ensure they contain data
print("Train split length:", len(Tinyds['train']))
print("Validation split length:", len(Tinyds['validation']))


Train split example: {'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}
Validation split example: {'text': 'Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."\n\nAfter playing with the car, Kitty and Spot felt thirsty. They found

In [9]:
from datasets import load_dataset


# Get the text from both the train and validation splits
train_texts = Tinyds['train']['text']  # Assuming 'text' is the column name
validation_texts = Tinyds['validation']['text']

# Check the first 3 texts in the train and validation splits
print("First 3 train texts:", train_texts[:3])
print("First 3 validation texts:", validation_texts[:3])

# Concatenate the texts from train and validation splits and also keep then separate in txt file

all_texts = train_texts + validation_texts

# Save a smaller portion to a .txt file to verify output
with open('tinystories_trainset100.txt', 'w') as f:
    for story in train_texts[:100]:  # Writing only the first 1ß0 stories
        f.write(story + '\n')  # Each story on a new line



First 3 train texts: ['One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.', 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should return True


2.4.1
True


In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("calum/tinystories-gpt2-3M")
model = AutoModelForCausalLM.from_pretrained("calum/tinystories-gpt2-3M")









#This isnt necesssary as we can just load the tokenizer through the pipeline, the   plan before was to create our own tokenizer
#import sentencepiece as spm

# Define paths and parameters
#input_text = r"C:\Users\Daniel\Documents\Daniel_things\Universität_Halle\Data_Mining_und_maschinelle_Lernen\FinalProect\CHASESQLTtry2\Ngrams\tinystories_trainset100.txt"  # Path to the prepared text file
#model_prefix = 'tinystories_tokenizer'  # Prefix for model files
#vocab_size = 32768  # Vocabulary size (same as in Nguyen's model)

# Train SentencePiece tokenizer
#spm.SentencePieceTrainer.Train(
   #input=input_text,
   #model_prefix=model_prefix,
   # vocab_size=vocab_size,
   #character_coverage=0.9995,  # Covers almost all characters
   #model_type="unigram",  # Use "unigram" model type, or try "bpe" or "word" if needed
   # user_defined_symbols=["<|endoftext|>"]  # Add special tokens if needed
#)

#print("Tokenizer training complete!")


C:\Users\Daniel\anaconda3\envs\fresh_huggingface\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Daniel\anaconda3\envs\fresh_huggingface\lib\site-packages\huggingface_hub-0.29.1-py3.8.egg\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
C:\Users\Daniel\anaconda3\envs\fresh_huggingface\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\Daniel\anaconda3\envs\fresh_huggingface\lib\site-packages\transformers\modeling_utils.py:488: FutureWarning: 

In [7]:
from datasets import load_dataset
import torch

# Load dataset
Tinyds = load_dataset("roneneldan/TinyStories")  # Ensure dataset is loaded
train_texts = Tinyds["train"]["text"]
val_texts = Tinyds["validation"]["text"]

# Define a padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS as PAD
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize model embedding to accommodate new token if added
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = Tinyds["train"].map(tokenize_function, batched=True)
val_dataset = Tinyds["validation"].map(tokenize_function, batched=True)

# Prepare for training (subset to speed up)
train_dataset = train_dataset.shuffle().select(range(10000))
val_dataset = val_dataset.shuffle().select(range(1000))

# Define DataLoader for batching
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)

# Move model to GPU
model.to("cuda")

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        inputs = batch["input_ids"].to("cuda")
        labels = batch["labels"].to("cuda")
        optimizer.zero_grad()

        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} completed with loss: {loss.item()}")


Using pad_token, but it is not set yet.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
Map: 100%|██████████████████████████████████████████████████████████████| 21990/21990 [00:08<00:00, 2577.62 examples/s]


AttributeError: 'list' object has no attribute 'to'

In [9]:
from datasets import load_dataset
import torch

# Load model and tokenizer (missing in your original code)
model_name = "gpt2-3M"  # or whatever model you're using

# Load dataset
train_texts = Tinyds["train"]["text"]
val_texts = Tinyds["validation"]["text"]

# Define a padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize model embedding to accommodate new token if added
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset and prepare labels for causal language modeling
def tokenize_function(examples):
    # Tokenize the texts
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    
    # For causal language modeling, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

# Apply tokenization with proper PyTorch tensors
train_dataset = Tinyds["train"].map(
    tokenize_function, 
    batched=True, 
    remove_columns=["text"]  # Remove original text column
)
val_dataset = Tinyds["validation"].map(
    tokenize_function, 
    batched=True, 
    remove_columns=["text"]
)

# Convert to PyTorch dataset format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Prepare for training (subset to speed up)
train_dataset = train_dataset.shuffle(seed=42).select(range(10000))
val_dataset = val_dataset.shuffle(seed=42).select(range(1000))

# Define DataLoader for batching
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_dataloader):
        # Move batch to device
        inputs = {k: v.to(device) for k, v in batch.items()}
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Print progress
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")
    
    # Print epoch summary
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} completed with average loss: {avg_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("tinystories_finetuned3pochs")
tokenizer.save_pretrained("tinystories_finetuned3epochs")

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
Map: 100%|██████████████████████████████████████████████████████████████| 21990/21990 [00:06<00:00, 3630.51 examples/s]


Epoch 1, Batch 100, Loss: 1.3375
Epoch 1, Batch 200, Loss: 0.8734
Epoch 1, Batch 300, Loss: 1.1576
Epoch 1, Batch 400, Loss: 1.1316
Epoch 1, Batch 500, Loss: 1.1082
Epoch 1, Batch 600, Loss: 1.2431
Epoch 1, Batch 700, Loss: 1.1271
Epoch 1, Batch 800, Loss: 1.5148
Epoch 1, Batch 900, Loss: 1.2944
Epoch 1, Batch 1000, Loss: 1.6702
Epoch 1, Batch 1100, Loss: 1.0314
Epoch 1, Batch 1200, Loss: 1.3085
Epoch 1, Batch 1300, Loss: 1.1116
Epoch 1, Batch 1400, Loss: 0.9005
Epoch 1, Batch 1500, Loss: 1.0981
Epoch 1, Batch 1600, Loss: 1.2579
Epoch 1, Batch 1700, Loss: 0.9629
Epoch 1, Batch 1800, Loss: 0.9809
Epoch 1, Batch 1900, Loss: 1.3713
Epoch 1, Batch 2000, Loss: 1.0438
Epoch 1, Batch 2100, Loss: 1.4486
Epoch 1, Batch 2200, Loss: 1.0999
Epoch 1, Batch 2300, Loss: 1.0341
Epoch 1, Batch 2400, Loss: 0.9450
Epoch 1, Batch 2500, Loss: 0.8611
Epoch 1 completed with average loss: 1.1145
Epoch 2, Batch 100, Loss: 1.2544
Epoch 2, Batch 200, Loss: 0.9298
Epoch 2, Batch 300, Loss: 0.8169
Epoch 2, Batch 4

('tinystories_finetuned3epochs\\tokenizer_config.json',
 'tinystories_finetuned3epochs\\special_tokens_map.json',
 'tinystories_finetuned3epochs\\vocab.json',
 'tinystories_finetuned3epochs\\merges.txt',
 'tinystories_finetuned3epochs\\added_tokens.json',
 'tinystories_finetuned3epochs\\tokenizer.json')

In [2]:
#from transformers import AutoTokenizer

# Load the GPT-2 (3M) tokenizer
#tokenizer = AutoTokenizer.from_pretrained("calum/tinystories-gpt2-3M")

# Test text
text = "This is a test"

# Encode: text -> token pieces & IDs
token_pieces = tokenizer.tokenize(text)  # Subword tokens
token_ids = tokenizer.encode(text)  # Token IDs

print("Token Pieces:", token_pieces)
print("Token IDs:", token_ids)

# Decode: tokens -> text
decoded_text = tokenizer.decode(token_ids)
print("Decoded Text:", decoded_text)

Token Pieces: ['This', 'Ġis', 'Ġa', 'Ġtest']
Token IDs: [1212, 318, 257, 1332]
Decoded Text: This is a test
